# 실력 긍부정

In [93]:
# 실력 긍부정 데이터 불러오기
import pandas as pd

sentence_df = pd.read_csv('./level-lable.csv', header=0, sep=',')
sentence_df.head(3)

,sentence,lable
0,"카오스, 아키에이지, 2012년~2014년 리그 오브 레전드, 2016년 오버워치,...",1
1,어떤 게임이든지 파고들기 시작하면 최상위권을 달성하며 엄청난 실력의 재능충 모습을 ...,1
2,게임 센스가 매우 뛰어난 편이며 화려한 플레이가 자주 나오는 편이다.,1


In [94]:
# 전처리 1
import re

sentence_df['sentence'] = sentence_df['sentence'].apply(lambda x : re.sub(pattern="[^ㄱ-ㅎ 가-힣 a-z A-Z]", repl=" ", string=str(x)))

sentence_df.head(10)

,sentence,lable
0,카오스 아키에이지 년 년 리그 오브 레전드 년 오버워치 ...,1
1,어떤 게임이든지 파고들기 시작하면 최상위권을 달성하며 엄청난 실력의 재능충 모습을 ...,1
2,게임 센스가 매우 뛰어난 편이며 화려한 플레이가 자주 나오는 편이다,1
3,이때는 잿불거인이 나타나기 전으로 우디르가 고인챔이라는 인식이 강해 천상계에서 아무...,1
4,당연히 천상계에선 원탑 우디르 유저,1
5,오버워치와 비슷한 FPS게임에 재능을 보이는 파카답게 솔로 스쿼드를 가리지 않고 엄...,1
6,롤 PD들이 파카 방송에 모여 스쿼드를 자주 하였는데 PD들 한두 명이 킬 킬...,1
7,플레이 스타일은 피지컬과 반응속도로 찍어누르는 재능충의 표본,1
8,뛰어난 로지컬과 상황 판단력으로 각을 재고 들어가는 타입이 아니라 일단 몸을 던지며...,1
9,의 상황에서도 본인의 피지컬을 신뢰하며 절대 빼지 않고 몸을 던진다,1


In [82]:
# 명사 추출 ( 안하는게 성능 더 좋을수도)

from konlpy.tag import Hannanum

hannanum = Hannanum()
i = 0
for sentence in sentence_df['sentence']:
    original = sentence
    string2noun = hannanum.nouns(original)
    noun2string = " ".join(string2noun)
    sentence_df['sentence'][i] = noun2string
    i += 1

sentence_df.head(10)

,sentence,lable
0,카오스 아키 년 년 리그 오브 레전드 년 오버워치 년 년 배틀그라운드 등 해 인기 ...,1
1,게임 시작 최상위권 달성 실력 재능충 모습 보,1
2,게임 센스 편 화 플레이 편,1
3,이때 잿불거인 전 우디르 고인챔 식 천상계 아무 우디르 우디르 승률 어마어마 수치 ...,1
4,천상계 원탑 우디르 유저,1
5,오버워치 비슷 재능 파카답게 솔 스쿼드,1
6,롤 파카 방송 스쿼드 들 한두 명 킬 멱살잡 우승 버스 기사 역할,1
7,플레이 스타일 피지컬 반응속도 찍어누르 재능충 표본,1
8,로지컬 상황 판단력 각 재고 타입 몸 공격적 포지셔닝,1
9,의 상황 본인 피지컬 신뢰 몸,1


In [95]:
# train-test set 나누기

from sklearn.model_selection import train_test_split

class_df = sentence_df['lable']
feature_df = sentence_df.drop(['lable'],axis=1, inplace=False)



X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.2, random_state=3)

X_train.shape, X_test.shape

((113, 1), (29, 1))

In [96]:
# 로지스틱 회귀 + countVectorize


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(ngram_range=(1,2))),
    ('lr_clf',LogisticRegression(C=10))

])

pipeline.fit(X_train['sentence'], y_train)
pred = pipeline.predict(X_test['sentence'])
pred_probs = pipeline.predict_proba(X_test['sentence'])[:,1]

print('예측 정확도는 {0:.4f}, ROC_AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))

cnt = 0
for review in X_test['sentence']:
    if pred[cnt] == 1:
        sentiment = 'positive'
    else:
        sentiment = 'negative'

    cnt += 1
    if cnt == 100:
        break

    print('댓글 :{0} 는 {1}로 예상됩니다'.format(review, sentiment))

예측 정확도는 0.8276, ROC_AUC는 0.9167
댓글 :상당히 계산적이고 상대방과의 대결에서 최선의 이익을 추구하는 플레이를 찾아내는 능력이 뛰어나다   는 negative로 예상됩니다
댓글 :롤 PD들이 파카 방송에 모여 스쿼드를 자주 하였는데 PD들 한두 명이  킬   킬을 하거나 빨리 죽어서 없더라도 멱살잡고 우승시켜주는 버스 기사 역할을 하였다  는 positive로 예상됩니다
댓글 :이때는 잿불거인이 나타나기 전으로 우디르가 고인챔이라는 인식이 강해 천상계에서 아무도 우디르를 쓰지 않았으나 우디르   판 승률     KDA    이라는 어마어마한 수치로 챌린저에 입성했다  는 positive로 예상됩니다
댓글 :LCK 관계자에게도 평판이 매우 높은데 이는 특히 게임 해설가 김동준의 언급처럼 단 한 번도 자신 실력에 대해서 자기 입으로 뽐낸 적은 없지만 이후 다른 사람이 내린 평가를 통해 역사상 최고 미드 라이너이자 역대 최고 리그 오브 레전드 프로게이머로 불리게 되었다는 점을 근거로 들 수 있다 는 positive로 예상됩니다
댓글 :   승  패 승률   로 놀라운 성적을 보여주고있다  는 positive로 예상됩니다
댓글 :슈퍼 익스퍼트 클리어를 대부분 성공할 정도로 컨트롤이 좋다  는 positive로 예상됩니다
댓글 :하지만 트페의 승률은 여전히     대이며 KDA도  점대 후반으로 시즌   트페의 선두주자라는 것을 다시 한번 증명하고 있다  는 positive로 예상됩니다
댓글 :오버워치와 비슷한 FPS게임에 재능을 보이는 파카답게 솔로 스쿼드를 가리지 않고 어마한 에임을 보여주고 있다  는 negative로 예상됩니다
댓글 :끝을 알 수 없는 암울한 피지컬을 가졌지만 근성으로 극복하며 마침내 정복해냈다  는 positive로 예상됩니다
댓글 :게임을 보면 막상 전술 자체는 그렇게까지 잘 짜지는 못하는 모습이 있다  는 negative로 예상됩니다
댓글 :풍월량 본인은 자신은 게임을 잘하거나 적어도 평균이라고 주장하나 주 시청자들에

In [97]:
# 로지스틱 회귀 + Tf-idf


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(ngram_range=(1,2))),
    ('lr_clf',LogisticRegression(C=10))

])

pipeline.fit(X_train['sentence'], y_train)
pred = pipeline.predict(X_test['sentence'])
pred_probs = pipeline.predict_proba(X_test['sentence'])[:,1]

print('예측 정확도는 {0:.4f}, ROC_AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))

cnt = 0
for review in X_test['sentence']:
    if pred[cnt] == 1:
        sentiment = 'positive'
    else:
        sentiment = 'negative'

    cnt += 1
    if cnt == 100:
        break

    print('댓글 :{0} 는 {1}로 예상됩니다'.format(review, sentiment))

예측 정확도는 0.8276, ROC_AUC는 0.9020
댓글 :상당히 계산적이고 상대방과의 대결에서 최선의 이익을 추구하는 플레이를 찾아내는 능력이 뛰어나다   는 negative로 예상됩니다
댓글 :롤 PD들이 파카 방송에 모여 스쿼드를 자주 하였는데 PD들 한두 명이  킬   킬을 하거나 빨리 죽어서 없더라도 멱살잡고 우승시켜주는 버스 기사 역할을 하였다  는 positive로 예상됩니다
댓글 :이때는 잿불거인이 나타나기 전으로 우디르가 고인챔이라는 인식이 강해 천상계에서 아무도 우디르를 쓰지 않았으나 우디르   판 승률     KDA    이라는 어마어마한 수치로 챌린저에 입성했다  는 positive로 예상됩니다
댓글 :LCK 관계자에게도 평판이 매우 높은데 이는 특히 게임 해설가 김동준의 언급처럼 단 한 번도 자신 실력에 대해서 자기 입으로 뽐낸 적은 없지만 이후 다른 사람이 내린 평가를 통해 역사상 최고 미드 라이너이자 역대 최고 리그 오브 레전드 프로게이머로 불리게 되었다는 점을 근거로 들 수 있다 는 positive로 예상됩니다
댓글 :   승  패 승률   로 놀라운 성적을 보여주고있다  는 positive로 예상됩니다
댓글 :슈퍼 익스퍼트 클리어를 대부분 성공할 정도로 컨트롤이 좋다  는 positive로 예상됩니다
댓글 :하지만 트페의 승률은 여전히     대이며 KDA도  점대 후반으로 시즌   트페의 선두주자라는 것을 다시 한번 증명하고 있다  는 positive로 예상됩니다
댓글 :오버워치와 비슷한 FPS게임에 재능을 보이는 파카답게 솔로 스쿼드를 가리지 않고 어마한 에임을 보여주고 있다  는 positive로 예상됩니다
댓글 :끝을 알 수 없는 암울한 피지컬을 가졌지만 근성으로 극복하며 마침내 정복해냈다  는 positive로 예상됩니다
댓글 :게임을 보면 막상 전술 자체는 그렇게까지 잘 짜지는 못하는 모습이 있다  는 negative로 예상됩니다
댓글 :풍월량 본인은 자신은 게임을 잘하거나 적어도 평균이라고 주장하나 주 시청자들에

In [39]:
# 또 다른 시도

import nltk
from konlpy.tag import Okt
t=Okt()

content_text = ""

for sentence in sentence_df['sentence']:
    content_text = content_text + sentence + "\n"

print(content_text)

카오스  아키에이지      년     년 리그 오브 레전드      년 오버워치      년     년 배틀그라운드 등 그 해에 가장 인기를 끌었던 게임마다 파고들면 최상위권을 달성할 정도로 여러 게임을 잘해서 유명해졌다 
어떤 게임이든지 파고들기 시작하면 최상위권을 달성하며 엄청난 실력의 재능충 모습을 보여줬다  
게임 센스가 매우 뛰어난 편이며 화려한 플레이가 자주 나오는 편이다  
이때는 잿불거인이 나타나기 전으로 우디르가 고인챔이라는 인식이 강해 천상계에서 아무도 우디르를 쓰지 않았으나 우디르   판 승률     KDA    이라는 어마어마한 수치로 챌린저에 입성했다 
당연히 천상계에선 원탑 우디르 유저 
오버워치와 비슷한 FPS게임에 재능을 보이는 파카답게 솔로 스쿼드를 가리지 않고 엄청난 에임을 보여주고 있다  
롤 PD들이 파카 방송에 모여 스쿼드를 자주 하였는데 PD들 한두 명이  킬   킬을 하거나 빨리 죽어서 없더라도 멱살잡고 우승시켜주는 버스 기사 역할을 하였다 
플레이 스타일은 피지컬과 반응속도로 찍어누르는 재능충의 표본  
뛰어난 로지컬과 상황 판단력으로 각을 재고 들어가는 타입이 아니라 일단 몸을 던지며 최대한 공격적인 포지셔닝을 잡는다 
   의 상황에서도 본인의 피지컬을 신뢰하며 절대 빼지 않고 몸을 던진다 
챔피언 폭도 진 같은 챔피언보다는 베인이나 트위치  코그모 같은 AD하이퍼캐리를 선호하며 그것이 아니라면 케이틀린이나 자야 같은 챔피언을 골라 라인전에서부터 압살을 하려 하는 챔피언을 좋아한다 
딸피로 살아나오는 플레이가 일품 
캐리력이 높은 브루저 챔피언으로 라인전을 터트리고 미드  바텀 순으로 똥을 퍼트리는 전형적인 탑신병자의 캐리 시나리오를 선호하는 모습 
 영웅 모두 화려한 실력으로 노마이크에 솔로큐임에도 불구하고 시즌  경쟁전   승  무   패       로   점을 달성했다 
하지만 예전과 같은 플레이를 보여주지 못해 팟수들의 실망을 자아내었다 
오버워치와 비슷한 FPS게임에 재능을 보이는 파카답게 솔로 스쿼드를

In [40]:
# 형태소 분석

tokens_ko = t.morphs(content_text)
tokens_ko

['카오스',
 '아키에이지',
 '년',
 '년',
 '리그',
 '오브',
 '레전드',
 '년',
 '오버',
 '워치',
 '년',
 '년',
 '배틀',
 '그',
 '라운드',
 '등',
 '그',
 '해',
 '에',
 '가장',
 '인기',
 '를',
 '끌었던',
 '게임',
 '마다',
 '파고들면',
 '최',
 '상위',
 '권',
 '을',
 '달성',
 '할',
 '정도',
 '로',
 '여러',
 '게임',
 '을',
 '잘',
 '해서',
 '유명해졌다',
 '어떤',
 '게임',
 '이든지',
 '파고들기',
 '시작',
 '하면',
 '최',
 '상위',
 '권',
 '을',
 '달성',
 '하며',
 '엄청난',
 '실력',
 '의',
 '재능',
 '충',
 '모습',
 '을',
 '보여줬다',
 '게임',
 '센스',
 '가',
 '매우',
 '뛰어난',
 '편이',
 '며',
 '화려한',
 '플레이',
 '가',
 '자주',
 '나오는',
 '편이',
 '다',
 '이',
 '때',
 '는',
 '잿불',
 '거인',
 '이',
 '나타나기',
 '전',
 '으로',
 '우디르',
 '가',
 '고인',
 '챔이라는',
 '인식',
 '이',
 '강해',
 '천상계',
 '에서',
 '아무',
 '도',
 '우디르',
 '를',
 '쓰지',
 '않았으나',
 '우디르',
 '판',
 '승률',
 'KDA',
 '이라는',
 '어마어마한',
 '수치',
 '로',
 '챌',
 '린저',
 '에',
 '입성',
 '했다',
 '당연히',
 '천상계',
 '에선',
 '원',
 '탑',
 '우디르',
 '유저',
 '오버',
 '워치',
 '와',
 '비슷한',
 'FPS',
 '게임',
 '에',
 '재능',
 '을',
 '보이는',
 '파카',
 '답',
 '게',
 '솔로',
 '스쿼드',
 '를',
 '가리지',
 '않고',
 '엄청난',
 '에',
 '임',
 '을',
 '보여주고',
 '있다',
 '롤',
 'PD

In [41]:
ko = nltk.Text(tokens_ko)
print(len(ko.tokens))
print(len(set(ko.tokens)))

3568
1390


In [42]:
#빈도수 확인 및 불용어 제거
ko.vocab().most_common(100)

[('을', 157),
 ('를', 76),
 ('의', 76),
 ('이', 71),
 ('에', 60),
 ('게임', 46),
 ('로', 41),
 ('가', 38),
 ('들', 31),
 ('실력', 30),
 ('으로', 29),
 ('은', 29),
 ('하는', 29),
 ('도', 28),
 ('에서', 27),
 ('한', 25),
 ('플레이', 23),
 ('했다', 20),
 ('것', 20),
 ('인', 19),
 ('는', 18),
 ('와', 18),
 ('있다', 18),
 ('적', 18),
 ('정도', 17),
 ('등', 16),
 ('때', 16),
 ('전', 16),
 ('과', 16),
 ('못', 16),
 ('선수', 16),
 ('그', 15),
 ('할', 15),
 ('최고', 15),
 ('챔피언', 14),
 ('시즌', 14),
 ('클리어', 14),
 ('컬', 13),
 ('하고', 13),
 ('말', 13),
 ('한다', 13),
 ('년', 12),
 ('이나', 12),
 ('수', 12),
 ('이다', 12),
 ('달성', 11),
 ('피지', 11),
 ('시청자', 11),
 ('여러', 10),
 ('월', 10),
 ('모습', 9),
 ('미드', 9),
 ('점', 9),
 ('나', 9),
 ('위', 9),
 ('능력', 9),
 ('리그', 8),
 ('오브', 8),
 ('하며', 8),
 ('매우', 8),
 ('다', 8),
 ('방송', 8),
 ('맵', 8),
 ('및', 8),
 ('자신', 8),
 ('사람', 8),
 ('레전드', 7),
 ('오버', 7),
 ('해', 7),
 ('잘', 7),
 ('하면', 7),
 ('엄청난', 7),
 ('승률', 7),
 ('게', 7),
 ('하', 7),
 ('같은', 7),
 ('이후', 7),
 ('까지', 7),
 ('또한', 7),
 ('일', 7),
 ('라는', 7),
 ('페이커', 7

In [43]:
stop_words = ['을', '를','의','이','에','로','가','들','으로','은','도','에서','했다','것','인','는','와','있다','때','과','그','하고','한다','년','수','이다']

tokens_ko = [each_word for each_word in tokens_ko
                if each_word not in stop_words]

ko = nltk.Text(tokens_ko)
ko.vocab().most_common(100)

[('게임', 46),
 ('실력', 30),
 ('하는', 29),
 ('한', 25),
 ('플레이', 23),
 ('적', 18),
 ('정도', 17),
 ('등', 16),
 ('전', 16),
 ('못', 16),
 ('선수', 16),
 ('할', 15),
 ('최고', 15),
 ('챔피언', 14),
 ('시즌', 14),
 ('클리어', 14),
 ('컬', 13),
 ('말', 13),
 ('이나', 12),
 ('달성', 11),
 ('피지', 11),
 ('시청자', 11),
 ('여러', 10),
 ('월', 10),
 ('모습', 9),
 ('미드', 9),
 ('점', 9),
 ('나', 9),
 ('위', 9),
 ('능력', 9),
 ('리그', 8),
 ('오브', 8),
 ('하며', 8),
 ('매우', 8),
 ('다', 8),
 ('방송', 8),
 ('맵', 8),
 ('및', 8),
 ('자신', 8),
 ('사람', 8),
 ('레전드', 7),
 ('오버', 7),
 ('해', 7),
 ('잘', 7),
 ('하면', 7),
 ('엄청난', 7),
 ('승률', 7),
 ('게', 7),
 ('하', 7),
 ('같은', 7),
 ('이후', 7),
 ('까지', 7),
 ('또한', 7),
 ('일', 7),
 ('라는', 7),
 ('페이커', 7),
 ('중', 7),
 ('워치', 6),
 ('재능', 6),
 ('챌', 6),
 ('롤', 6),
 ('거나', 6),
 ('라인', 6),
 ('력', 6),
 ('고', 6),
 ('모두', 6),
 ('에게', 6),
 ('\n', 6),
 ('대부분', 6),
 ('커리어', 6),
 ('번', 6),
 ('서', 6),
 ('시간', 6),
 ('컨트롤', 6),
 ('계속', 6),
 ('시야', 6),
 ('풍월량', 6),
 ('카오스', 5),
 ('권', 5),
 ('시작', 5),
 ('뛰어난', 5),
 ('솔로', 5),
 ('않고',

In [47]:
i = 0
for sentence in sentence_df['sentence']:
    morph_sentence = t.morphs(sentence)
    clean_sentence = [each_word for each_word in morph_sentence
                if each_word not in stop_words]
    new_sentence = " ".join(clean_sentence)
    sentence_df['sentence'][i] = new_sentence
    i += 1

sentence_df.head(10)

,sentence,lable
0,카오스 아키에이지 리그 오브 레전드 오버 워치 배틀 라운드 등 해 가장 인기 끌었던...,1
1,어떤 게임 이든지 파고들기 시작 하면 최 상위 권 달성 하며 엄청난 실력 재능 충 ...,1
2,게임 센스 매우 뛰어난 편이 며 화려한 플레이 자주 나오는 편이 다,1
3,잿불 거인 나타나기 전 우디르 고인 챔이라는 인식 강해 천상계 아무 우디르 쓰지 않...,1
4,당연히 천상계 에선 원 탑 우디르 유저,1
5,오버 워치 비슷한 FPS 게임 재능 보이는 파카 답 게 솔로 스쿼드 가리지 않고 엄...,1
6,롤 PD 들이 파카 방송 모여 스쿼드 자주 하였는데 PD 한두 명 킬 킬 하 거나 ...,1
7,플레이 스타일 피지 컬 반응속도 찍어 누르는 재능 충의 표본,1
8,뛰어난 로지 컬 상황 판단력 각 재고 들어가는 타입 아니라 일단 몸 던지며 최대한 ...,1
9,상황 에서도 본인 피지 컬 신뢰 하며 절대 빼지 않고 몸 던진다,1


In [104]:
# 모델 저장

import pickle

with open('pipe_tfidf.dat', 'wb') as fp:
    pickle.dump(pipeline, fp)

In [105]:
# 새로운 문장 예측함수

def using_model(string):
    import numpy as np
    with open('pipe_tfidf.dat', 'rb') as fp:
        pipeline = pickle.load(fp)

    text = [string]
    

    # 예측 스코어
    r1 = np.max(pipeline.predict_proba(text) * 100)
    
    # 예측 결과
    r2 = pipeline.predict(text)[0]
    

    if r2 == 1 :
        print('실력 좋다')
    else :
        print('실력 나쁘다')

    print('스코어 : %.3f '% r1)


using_model('진짜 존나 못한다')

실력 나쁘다
스코어 : 56.659 
